In [1]:
#//*** This thread helped with the connection
#//***https://stackoverflow.com/questions/37692780/error-28000-login-failed-for-user-domain-user-with-pyodbc

#//**** Update the ODBC Driver
#//**** https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15
import pyodbc
import datetime
import json
from datetime import date, timedelta
import pandas as pd
print(pyodbc.drivers())

from sklearn.feature_extraction.text import TfidfVectorizer
import time

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'tcp:OM-CASF-DB01' 
server = 'OM-CASF-DLSQL' 
# server = '10.218.97.2'
database = 'DaletDB' 

with open('./ignore_folder/misc.json') as f:
    data = json.loads(f.read())

username = data["user"] 
password = data["password"]
del data
#cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cnxn = pyodbc.connect('Trusted_Connection=yes;DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

['SQL Server', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client 11.0']


In [2]:
#//************************************************************
#//************************************************************
#//*** get all Rundowns in given Quarter
#//************************************************************
#//************************************************************

#//*** Table: titles
#//*** Search Titles by Date to get Rundown IDs
#//*** Titles Contains All titled Objects including scripts rundowns and MOS objects
#//*** title_type_id = 25 <--- Rundown Objects
#//*** duration > 900000 <--- Rundowns Longer than 15 minutes (15 * 60 Seconds * 1000 ms)
#//***                        Automatically filters out short content like cutins
title_cols = ['title_id','title_type_id','title','duration','start_date']
import datetime


tgt_year = 2021
quarter = "Q4"

#//*** Build the BETWEEN portion of the query based on quarter and YEAR

quarter_query = ""
if quarter == "Q1":
    quarter_query = f"'{tgt_year}-01-01T00:00:00' AND '{tgt_year}-03-31T23:59:59'"

elif quarter == "Q2":
    quarter_query = f"'{tgt_year}-04-01T00:00:00' AND '{tgt_year}-06-30T23:59:59'"

elif quarter == "Q3":
    quarter_query = f"'{tgt_year}-07-01T00:00:00' AND '{tgt_year}-09-30T23:59:59'"

elif quarter == "Q4":
    quarter_query = f"'{tgt_year}-09-01T00:00:00' AND '{tgt_year}-12-31T23:59:59'"

query = f"""
SELECT *
FROM titles 
WHERE title_type_id = 25 
    AND duration > 900000
    AND start_date BETWEEN {quarter_query}
    AND title <> 'Prepak'
    AND title <> 'Tool Kit'
    AND title <> 'Tricaster'
    AND title <> 'PRODUCER HOLD'
    AND title <> 'Promo'
    AND title <> 'Breaking News'
    AND title <> 'Dalet OD XPression'
    AND title <> 'PRACTICE'
    AND title <> '7AM DIGITAL'
"""


cursor.execute(query)
results = cursor.fetchall()
#results = cursor.fetchmany(100)

all_rundowns_df = pd.read_sql(query,cnxn)

print(all_rundowns_df.columns)

all_rundowns_df = all_rundowns_df[title_cols]

#//*** Filter out shows that SAY DO NOT USE
all_rundowns_df = all_rundowns_df[all_rundowns_df['title'].str.contains('DO NOT USE')==False]

#//*** Convert Start_Date to Date only
all_rundowns_df['start_date'] = all_rundowns_df['start_date'].apply(lambda x: x.date())

all_dates = all_rundowns_df['start_date'].unique()

#//*** Pick one day to test with
one_day_df = all_rundowns_df[all_rundowns_df['start_date'] == all_dates[-3]]


print("Quarterly Shows : ", len(all_rundowns_df))
print("Quarterly Show Hours: ", all_rundowns_df['duration'].sum() / 60000 / 60)
all_rundowns_df.iloc[-40:]
all_rundowns_df

Index(['title_id', 'title_type_id', 'title_interface_id', 'user_id', 'title',
       'interpret', 'author', 'client', 'duration', 'is_online', 'is_recorded',
       'record_date', 'start_date', 'end_date', 'kill_date', 'audio_duration',
       'use_manual_duration', 'soundfile_id', 'year', 'keywords',
       'site_origin', 'no_overwrite', 'is_rotational_cart', 'replacement',
       'last_words', 'is_opener', 'weight', 'beats_pm', 'master_record',
       'compagny_id', 'compagny_disp_name', 'album_disp_name', 'package_id',
       'album_id', 'song_id', 'day_part_rest_id', 'keep_date'],
      dtype='object')
Quarterly Shows :  764
Quarterly Show Hours:  578.8850675


,title_id,title_type_id,title,duration,start_date
0,32556826,25,5PM Weekday,1680011,2021-09-01
1,32557644,25,6PM Weekday,3515979,2021-09-01
2,32560085,25,11PM WEEKDAY,2113044,2021-09-01
3,32564498,25,5AM Weekday,3435031,2021-09-01
4,32566026,25,6AM Weekday,3630093,2021-09-01
...,...,...,...,...,...
765,34016213,25,11PM WEEKDAY,2113044,2021-12-01
766,34016510,25,3PM Getting Answers,1799998,2021-12-01
767,34016644,25,4PM Weekday,3674003,2021-12-01
768,34017262,25,5PM Weekday,1680011,2021-12-01


In [3]:
#//***************************************************************************
#//***************************************************************************
#//*** Scrape All Stories For the Given Quarter and export to XLS
#//***************************************************************************
#//***************************************************************************
qtr_df = pd.DataFrame()
for date in all_rundowns_df['start_date'].unique():
    one_day_df = all_rundowns_df[all_rundowns_df['start_date'] == date]
    print(date, "/", all_rundowns_df['start_date'].unique()[-1])
    for row in one_day_df.iterrows():
        loop_rundown = row[1]
        loop_title_id = loop_rundown['title_id']
        #print("Rundown ID: ", loop_title_id, loop_rundown['title'])

        #print(loop_rundown)
        #//*** Get the Blocks associated with the Selected Rundown
        query = f"""
        SELECT block_id
        FROM items 

        WHERE clock_id = '{loop_title_id}'
        """
        loop_blocks = pd.read_sql(query,cnxn)['block_id'].unique()

        #//*** Find the A-Block of Selected Rundown
        #//**** Block Query gets just the A BLOCK from the Rundown Block
        #//*** Combines all Blocks into a single query and returns only the A-Block
        block_query = ""
        for block_id in loop_blocks:
            if block_id == loop_blocks[0]:
                block_query += f"(block_id='{block_id}' "
            else:
                block_query += f"OR block_id='{block_id}' "
        block_query += ") AND title='A BLOCK'"

        query = f"""
        SELECT *
        FROM blocks 
        WHERE {block_query}
        """

        tdf = pd.read_sql(query,cnxn)
        #//*** A Block of Selected Rundown
        if len(tdf) == 0:
            print("================")
            print("NO A-Block Found")
            print("================")
            print(one_day_df)
            continue

        a_block = tdf["block_id"].values[0]
        #print("A Block:",a_block)

        #//******************************
        #//*** Get A-Block Story Titles 
        #//******************************
        query = f"""
        SELECT title_id
        --SELECT block_id,item_id,title_id
        FROM spots 
        WHERE block_id = '{a_block}'
        """

        title_id_list = pd.read_sql(query,cnxn)['title_id'].values


        #//****************************************************
        #//*** Get title_id, Story Slug, and Duration
        #//*** Only get Stories with Greater than 0 Duration
        #//****************************************************

        #//*** Build Single Query to get all Titles from Selected Rundown

        title_id_query = ""
        for title_id in title_id_list:
            if title_id == title_id_list[0]:
                title_id_query += f"title_id='{title_id}' "
            else:
                title_id_query += f"OR title_id='{title_id}' "

        query = f"""
        SELECT title_id,title_type_id,title,duration
        FROM titles 
        WHERE {title_id_query}
        """

        titles_df = pd.read_sql(query,cnxn)

        #//*** Remove Stories with Zero Duration
        #//*** This Removes stories with no text
        titles_df = titles_df[titles_df['duration'] > 0]

        #//*** Remove stories with the word tease in slug
        titles_df = titles_df[titles_df['title'].str.lower().str.contains('tease') == False]
        #print(titles_df)
        #//*** Get StoryBody Text
        story_id_list = titles_df['title_id'].values
        story_id_query = ""
        for story_id in story_id_list:
            if story_id == story_id_list[0]:
                story_id_query += f"TitleId='{story_id}' "
            else:
                story_id_query += f"OR TitleId='{story_id}' "

        query = f"""
        SELECT TitleId,StoryText
        FROM StoryContent
        WHERE {story_id_query}
        """

        text_df = pd.read_sql(query,cnxn)
        #print(text_df)

        #//*** Build Stories_df List of all Stories in show with Text and Title
        #//*** Merge titles_df and text_df 
        stories_df = titles_df.merge(text_df,left_on='title_id',right_on='TitleId')

        #//*** Delete the duplicate column
        del stories_df['TitleId']

        #//********************
        #//*** Clean the Text
        #//********************

        #//*** Remove Brackets [[ ]]
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('\[\[\*\*\*.*?\*\*\*\]\]','\\n',regex=True)

        #//*** Remove paranthesis (( ))
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('\(\(.*?\)\)','\\n',regex=True)

        #//*** Covert \r\n to \n
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('\\r\\n','\\n',regex=True)

        #//*** Convert Multiple \n to single \n
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('\\n\\W*\\n','\\n',regex=True)

        #//*** Delete leading \n
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('^\\W*\\n','',regex=True)
        #print(stories_df.iloc[7]['StoryText'])

        #//*** Rename title column to storyslug
        stories_df.columns = ['storyslug' if x=='title' else x for x in list(stories_df.columns)]


        #//*** Start Date, Title, from the current rundown. This Column will have the same value for all rows.
        static_cols = ['start_date','title']

        for col in static_cols:
            stories_df[col] = loop_rundown[col]

        #//*** Shift the static_cols to the beginning of the column list for readability
        cols = (static_cols + list(stories_df.columns))[:(len(static_cols)*-1)]

        stories_df = stories_df[cols]


        #//*** Rename title column to storyslug
        stories_df.columns = ['rundown' if x=='title' else x for x in list(stories_df.columns)]


        #print(cols)
        #print(stories_df)
        qtr_df = pd.concat([qtr_df,stories_df],ignore_index=True)
        #//*** End Get Single Rundown Stories
    
    #//*** End Each Day of Rundowns

#//*** Remove Stories with 0 length Text
qtr_df['StoryText'] = qtr_df['StoryText'].astype(str)
qtr_df['length'] = qtr_df['StoryText'].apply(lambda x : len(x))
qtr_df = qtr_df[qtr_df['length'] > 20]

if 'title_type_id' in qtr_df.columns:
    del qtr_df['title_type_id']
    
if 'length' in qtr_df.columns:
    del qtr_df['length']
tdf = qtr_df
tdf['duration'] = (tdf['duration'] /1000).astype(int)    

#qtr_df.to_csv("2021_Q4_Stories.csv")  


# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'{tgt_year}_{quarter}_Stories.xlsx', engine='xlsxwriter')
qtr_df.to_excel(writer,sheet_name='sheet1')
writer.save()

qtr_df
#print("Done")

    
    
    
    

2021-09-01 / 2021-12-01
2021-09-02 / 2021-12-01
2021-09-03 / 2021-12-01
2021-09-04 / 2021-12-01
2021-09-05 / 2021-12-01
2021-09-06 / 2021-12-01
2021-09-07 / 2021-12-01
2021-09-08 / 2021-12-01
2021-09-09 / 2021-12-01
2021-09-10 / 2021-12-01
2021-09-11 / 2021-12-01
2021-09-12 / 2021-12-01
2021-09-13 / 2021-12-01
2021-09-14 / 2021-12-01
2021-09-15 / 2021-12-01
2021-09-16 / 2021-12-01
2021-09-17 / 2021-12-01
2021-09-18 / 2021-12-01
2021-09-19 / 2021-12-01
2021-09-20 / 2021-12-01
2021-09-21 / 2021-12-01
2021-09-22 / 2021-12-01
2021-09-23 / 2021-12-01
2021-09-24 / 2021-12-01
2021-09-25 / 2021-12-01
2021-09-26 / 2021-12-01
2021-09-27 / 2021-12-01
2021-09-28 / 2021-12-01
2021-09-29 / 2021-12-01
2021-09-30 / 2021-12-01
2021-10-01 / 2021-12-01
2021-10-02 / 2021-12-01
2021-10-03 / 2021-12-01
2021-10-04 / 2021-12-01
2021-10-05 / 2021-12-01
2021-10-06 / 2021-12-01
2021-10-07 / 2021-12-01
2021-10-08 / 2021-12-01
2021-10-09 / 2021-12-01
2021-10-10 / 2021-12-01
2021-10-11 / 2021-12-01
2021-10-12 / 202

,start_date,rundown,title_id,storyslug,duration,StoryText
7,2021-09-01,5PM Weekday,32556910,5PCOLDOPEN 1,9,[[QUOTE: we're going to start to see the conta...
8,2021-09-01,5PM Weekday,32556916,5P HELLO,7,"Good Evening, I'm Ama Daetz.\nand I'm Dan Ash..."
9,2021-09-01,5PM Weekday,32556923,5P HELLO FRAMES,2,"Good Evening, I'm Ama Daetz.\nand I'm Dan Ash..."
13,2021-09-01,5PM Weekday,32570152,5P DIXIE FIRE OC-VO,24,The massive Dixie Fire is showing no signs of ...
16,2021-09-01,5PM Weekday,32570960,5P TAHOE FIRE PKG,65,HERE ARE SOME OF THE WORDS WE HAVE HEAR FROM F...
...,...,...,...,...,...,...
23123,2021-12-01,4PM Weekday,34016717,HELLO (DAN),7,Good afternoon\nI'm Kristen Sze...\nAnd I'm Da...
23124,2021-12-01,4PM Weekday,34016723,HELLO (Ama),7,Good afternoon\nI'm Larry Beil...\nAnd I'm Ama...
23125,2021-12-01,5PM Weekday,34017347,5P HELLO,7,"Good Evening, I'm Ama Daetz.\nand I'm Dan Ash..."
23126,2021-12-01,5PM Weekday,34017354,5P HELLO FRAMES,7,"Good Evening, I'm Ama Daetz.\nand I'm Dan Ash..."


In [4]:
qtr_df = pd.read_excel(f'{tgt_year}_{quarter}_Stories.xlsx')

In [5]:
"""
qtr_df = pd.read_excel(f'{tgt_year}_{quarter}_Stories.xlsx')

#//*** Trim unnamed First Column (original Index)
del qtr_df[qtr_df.columns[0]]

print(f"Begin tfidf....")
start_time = time.time()
#tfidf = TfidfVectorizer(ngram_range=(1,3),max_features=10000)
tfidf = TfidfVectorizer(ngram_range=(1,1), stop_words={'english'},)
tfidf_matrix = tfidf.fit_transform(qtr_df['StoryText'])
print(f"tfidf Built: {round(time.time()-start_time,2)}s")


qtr_df = pd.merge(qtr_df, pd.DataFrame(tfidf_matrix,columns=["tfidf"]), left_index=True, right_index=True)
qtr_df

#all_text = remove_stop_words(tokenize_series(mr_clean_text(pd.Series( ' '.join(list(qtr_df["StoryText"]))))))

"""
print()

In [6]:
all_text =pd.Series(( ' '.join(list(qtr_df["StoryText"]))))

custom_stop_words = ['quote','the','abc7','news','new',
                     'to','get','live','say','says','be','bay','be','0','1','2','3','4','5','6','7','8','9',
                     'day','is','in','en','are','th','of','im','it','sot','not','that','reporter'
                    ]
#custom_stop_words = []
print(len(all_text[0]))
for word in custom_stop_words:
    all_text[0] = all_text[0].replace(word, '')

print(len(all_text[0]))

#tfidf = TfidfVectorizer(ngram_range=(1,3),max_features=10000)
#tfIdfVectorizer=TfidfVectorizer(ngram_range=(1,3),use_idf=True)
tfIdfVectorizer=TfidfVectorizer(ngram_range=(4,4),use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(all_text)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(75))

5556911
4664968
                                      TF-IDF
and wherever you stream             0.287185
on abc hulu live                    0.268587
hulu live and wherever              0.268587
live and wherever you               0.268587
abc hulu live and                   0.268155
you re watchg abc                   0.184248
re watchg abc news                  0.125860
watchg abc news at                  0.125860
here on abc hulu                    0.089962
abc dot com slash                   0.080879
dot com slash vacce                 0.070066
dot com slash vote                  0.069201
com slash vacce and                 0.067471
vacce and click on                  0.067039
slash vacce and click               0.067039
sze you re watchg                   0.065741
krt sze you re                      0.065309
abc news dot com                    0.064011
watchg abc morngs on                0.058389
morngs on abc hulu                  0.058389
re watchg abc morngs                0.0

In [7]:
qtr_df = pd.read_excel(f'{tgt_year}_{quarter}_Stories.xlsx')

#//*** Trim unnamed First Column (original Index)
del qtr_df[qtr_df.columns[0]]

qtr_df = qtr_df[qtr_df['StoryText'].str.lower().str.contains("major fires across the state and track")==False]

#all_text =pd.Series(( ' '.join(list(qtr_df["StoryText"]))))
all_text = []

ngrams = 10

custom_stop_words = ['sot in person interview', '0','1','2','3','4','5','6','7','8','9','\'','"','\n','\t',',','.',',','-',':',
                    'sot in person', ]
custom_stop_words = []

for group in qtr_df.groupby('start_date'):
    loop_text = ' '.join(list(group[1]['StoryText']))
    
    for word in custom_stop_words:
        loop_text = loop_text.replace(word," ")
    
    all_text.append(loop_text)
    
tfIdfVectorizer=TfidfVectorizer(ngram_range=(ngrams,ngrams),use_idf=True,smooth_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(all_text)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(len(df))
print (df[:100].index)
#print (df[1000:1100])


623148
Index(['who are part of the group asking for higher pay',
       'part of the group asking for higher pay and better',
       'are part of the group asking for higher pay and',
       'of the group asking for higher pay and better covid',
       'the first time ever the national weather service issuing flash',
       'ever the national weather service issuing flash flood emergency for',
       'first time ever the national weather service issuing flash flood',
       'for the first time ever the national weather service issuing',
       'and for the first time ever the national weather service',
       'time ever the national weather service issuing flash flood emergency',
       'for liz kreutz let start with quick look at the',
       'in for liz kreutz let start with quick look at',
       'shaping up to be historic abc7 news reporter cornell barnard',
       'officials say voter turnout is already shaping up to be',
       'is already shaping up to be historic abc7 news repo

In [8]:
qtr_df = pd.read_excel(f'{tgt_year}_{quarter}_Stories.xlsx')

#//*** Trim unnamed First Column (original Index)
del qtr_df[qtr_df.columns[0]]

qtr_df = qtr_df[qtr_df['StoryText'].str.lower().str.contains("major fires across the state and track")==False]

#all_text =pd.Series(( ' '.join(list(qtr_df["StoryText"]))))
all_text = []

ngrams = 2

custom_stop_words = ['sot in person interview', '0','1','2','3','4','5','6','7','8','9','\'','"','\n','\t',',','.',',','-',':',
                    'sot in person', ]
custom_stop_words = ['4p','5a','5p','6a','6p','8a','8p','9a','9p','11a','11p',
                     'anchor','acoldopen',
                     'coldopen',
                     
                     'fs','fr','frames','fls','fr1',
                     'hello',
                     'int','intro',
                     'liz',
                     'oc',
                     'pkg','preshow','qa1',
                     'ra','rem',
                     'setup','sotvo','sv','sv1',
                     'toss','tag','t1',
                     'update',
                     'vo',
                     'wrap',
                    ]
remove_characters = ["-",":","\xa0","+"]
all_text = []

for group in qtr_df.groupby('start_date'):
    loop_text = ' '.join(list(group[1]['storyslug'])).lower().split(" ")

    for word in loop_text:
        for find in remove_characters:
            if find in word:
                #print(find,word)
                loop_text.remove(word)
            
    
    for word in custom_stop_words:
        while word in loop_text:
            loop_text.remove(word)
        
    loop_text = ' '.join(loop_text)
    all_text.append(loop_text)
    
tfIdfVectorizer=TfidfVectorizer(ngram_range=(2,2),use_idf=True,smooth_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(all_text)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(len(df))
print (df[:100])
print (df[100:200])
#print (df[1000:1100])


8953
                       TF-IDF
tahoe fire           0.415178
caldor fire          0.360921
fire tahoe           0.311384
strike vote          0.233538
mt lassen            0.222107
giants strike        0.210518
lassen fire          0.194344
voting centers       0.155692
northeast storms     0.151606
recall daily         0.151606
fire caldor          0.121011
vote giants          0.111054
fire ida             0.111054
fire latest          0.107881
fire cal             0.090963
newsom visit         0.090963
strike icu           0.090963
teamwork strike      0.090963
daily recall         0.090963
booster delay        0.083290
strike strike        0.083290
cal fire             0.083290
icu surge            0.083290
fire teamwork        0.083290
ida aftermath        0.073623
covid testing        0.067256
virus latest         0.062500
fire drew            0.060642
fire northeast       0.060642
gavin no             0.060642
testing booster      0.060642
fire mt              0.060642
north

In [9]:
freq = {}

for word in ' '.join(list(df.index)).split(' '):
    if word in freq.keys():
        freq[word] += 1
    else:
        freq[word] = 1

#sorted(month, key=numbermap.__getitem__)

reverseDict = {}
for key in sorted(freq, key=freq.get, reverse=True):
    print(key,freq[key])
    if freq[key] < 20:
        break
    #oldKey_newValue = key
    #newKey_oldValue = input_dict[key]

vax 323
covid 278
fire 240
weather 232
sf 155
cold 152
amy 145
recall 138
wx 138
shooting 130
open 118
oakland 117
booster 116
jobina 109
questions 102
vaccine 97
giants 92
newsom 91
biden 91
school 87
boosters 85
5pcoldopen 81
marin 76
sj 75
acold 74
cornell 74
and 72
travel 71
mandate 71
first 70
latest 69
rain 67
stephanie 66
psps 65
bay 65
atg 63
web 61
kid 60
webpc 60
oak 59
coronavirus 59
theranos 58
caldor 56
ryan 56
ax 55
kate 52
luz 52
push 52
kids 52
facebook 51
mask 51
shoot 51
headlines 49
crime 48
studio 48
holiday 48
kaiser 48
pfizer 48
vs 48
masks 47
atoss 46
txl 46
variant 45
vaccines 45
theft 43
alert 43
ca 42
san 42
to 42
app 42
storm 41
ctv 41
air 41
phil 40
coco 40
ousd 40
reax 40
omicron 40
pge 40
shot 40
t1 39
map 39
looting 39
drought 39
outage 39
coverage 38
cdc 38
water 36
county 36
tim 35
fda 35
new 34
union 34
bart 34
day 34
tracker 34
climate 33
warriors 33
880 33
opd 32
lyanne 32
preps 32
strike 31
team 31
outages 31
5atoss 30
santa 30
power 30
park 30
taho

In [10]:
freq = {}

for word in ' '.join(all_text).split(' '):
    if word in freq.keys():
        freq[word] += 1
    else:
        freq[word] = 1

#sorted(month, key=numbermap.__getitem__)

reverseDict = {}
for key in sorted(freq, key=freq.get, reverse=True):
    print(key,freq[key])
    #oldKey_newValue = key
    #newKey_oldValue = input_dict[key]
    if freq[key] < 20:
        break


vax 552
covid 376
fire 351
weather 267
open 263
cold 225
first 223
wx 218
studio 194
shooting 180
recall 174
 174
sf 170
amy 149
questions 149
bay 140
booster 129
traffic 127
school 119
oakland 113
kid 112
boosters 104
newsom 103
vaccine 102
giants 97
travel 96
jobina 94
mandate 93
marin 93
and 92
latest 90
biden 86
rain 82
caldor 81
kids 78
1 74
headlines 74
ax 74
atg 71
pfizer 71
psps 67
q 66
a 66
top 66
txl 65
quad 64
web 63
acold 63
storm 62
sj 62
mask 61
stephanie 60
webpc 59
push 59
crime 56
variant 56
880 54
5pcoldopen 53
day 53
kaiser 53
tracker 52
alert 52
holiday 51
app 50
air 49
theft 49
tahoe 48
q&a 48
cornell 48
shoot 48
climate 48
facebook 45
vaccines 45
t.1 45
ctv 44
coco 44
coronavirus 44
vs 44
east 44
strike 43
fda 43
luz 43
of 42
oak 42
county 42
union 41
ousd 40
power 40
outages 40
looting 40
ca 39
preps 39
omicron 39
theranos 38
pge 38
map 37
coverage 37
freeway 37
water 37
reax 37
outage 37
cal 36
the 36
week 36
san 35
masks 35
ryan 35
north 35
fleet 35
area 34
bar

In [11]:
qtr_df = pd.read_excel(f'{tgt_year}_{quarter}_Stories.xlsx')

#//*** Trim unnamed First Column (original Index)
del qtr_df[qtr_df.columns[0]]

qtr_df = qtr_df[qtr_df['StoryText'].str.lower().str.contains("major fires across the state and track")==False]

#all_text =pd.Series(( ' '.join(list(qtr_df["StoryText"]))))
all_text = []

ngrams = 10

custom_stop_words = ['sot in person interview', '0','1','2','3','4','5','6','7','8','9','\'','"','\n','\t',',','.',',','-',':',
                    'sot in person', ]
custom_stop_words = ['4p','5a','5p','6a','6p','8a','8p','9a','11a','11p',
                     'anchor','acoldopen',
                     'coldopen',
                     
                     'fs','fr','frames','fls','fr1',
                     'hello',
                     'int','intro',
                     'liz',
                     'oc',
                     'pkg','preshow','qa1',
                     'ra','rem',
                     'setup','sotvo','sv','sv1',
                     'toss','tag','t1',
                     'update',
                     'vo',
                     'wrap',
                    ]
remove_characters = ["-",":","\xa0","+"]
all_text = []

for group in qtr_df.groupby('start_date'):
    loop_text = ' '.join(list(group[1]['storyslug'])).lower().split(" ")

    for word in loop_text:
        for find in remove_characters:
            if find in word:
                #print(find,word)
                loop_text.remove(word)
            
    
    for word in custom_stop_words:
        while word in loop_text:
            loop_text.remove(word)
        
    loop_text = ' '.join(loop_text)
    all_text.append(loop_text)
    
tfIdfVectorizer=TfidfVectorizer(ngram_range=(ngrams,ngrams),use_idf=True,smooth_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(all_text)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(len(df))
print (df[:100])
#print (df[1000:1100])


17278
                                                      TF-IDF
caldor fire tahoe fire tahoe fire cal fire team...  0.148483
fire tahoe fire tahoe fire cal fire teamwork st...  0.148483
caldor fire caldor fire tahoe fire tahoe fire c...  0.148483
fire tahoe fire cal fire teamwork strike strike...  0.148483
tahoe fire tahoe fire cal fire teamwork strike ...  0.148483
fire caldor fire tahoe fire tahoe fire cal fire...  0.148483
fire cal fire teamwork strike strike icu surge ...  0.098989
voting centers voting centers scc voting center...  0.098989
fire teamwork strike strike icu surge covid tes...  0.098989
tahoe fire cal fire teamwork strike strike icu ...  0.098989
cal fire teamwork strike strike icu surge covid...  0.098989
centers voting centers scc voting centers newso...  0.098989
strike icu surge covid testing booster delay gi...  0.098989
strike strike icu surge covid testing booster d...  0.098989
teamwork strike strike icu surge covid testing ...  0.098989
strike vote 5atoss

In [12]:
tfIdfVectorizer=TfidfVectorizer(ngram_range=(2,2),use_idf=True,smooth_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(df.index)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

freq = {}

for word in ' '.join(list(df.index)).split(' '):
    if word in freq.keys():
        freq[word] += 1
    else:
        freq[word] = 1

#sorted(month, key=numbermap.__getitem__)

reverseDict = {}
for key in sorted(freq, key=freq.get, reverse=True):
    print(key,freq[key])
    #oldKey_newValue = key
    #newKey_oldValue = input_dict[key]

vax 323
covid 278
fire 240
weather 232
sf 155
cold 153
amy 145
recall 139
wx 138
shooting 128
open 118
oakland 117
booster 116
jobina 109
questions 102
vaccine 97
giants 92
newsom 92
biden 91
school 87
boosters 85
5pcoldopen 81
marin 76
sj 75
acold 74
cornell 74
and 72
travel 71
mandate 71
first 70
latest 69
rain 67
stephanie 66
bay 65
psps 65
atg 63
web 61
kid 60
webpc 60
oak 59
coronavirus 59
theranos 58
caldor 56
ryan 56
ax 55
kate 52
luz 52
push 52
kids 52
facebook 51
mask 51
shoot 51
headlines 49
holiday 48
crime 48
kaiser 48
pfizer 48
vs 48
studio 48
masks 47
atoss 46
txl 46
variant 45
vaccines 45
theft 43
alert 43
ca 42
san 42
to 42
app 42
ctv 41
air 41
storm 41
reax 40
coco 40
phil 40
ousd 40
omicron 40
pge 40
shot 40
drought 39
outage 39
looting 39
t1 39
map 39
coverage 38
cdc 38
county 36
water 36
tim 35
fda 35
new 34
union 34
bart 34
day 34
tracker 34
880 33
climate 33
warriors 33
opd 32
lyanne 32
preps 32
strike 31
team 31
outages 31
santa 30
5atoss 30
power 30
park 30
taho

alto 3
palisades 3
paintball 3
orphans 3
antiviral 3
pharmacy 3
garage 3
steam 3
imapct 3
sterling 3
van 3
valero 3
survey 3
vegas 3
blood 3
trials 3
vallejo 3
suit 3
doctor 3
uc 3
tribute 3
trip 3
levee 3
trend 3
tulare 3
ups 3
hiring 3
usc 3
vaccinations 3
vacancy 3
wellspring 3
webb 3
later 3
woods 3
immunize 3
spca 3
latino 3
block 3
waves 3
chinese 3
cliff 3
statue 3
benefits 3
arson 3
maritime 3
icu 3
laird 3
streets 3
bulldog 3
e40 3
shutdown 3
dashcam 3
flex 3
geary 3
tolls 3
tok 3
fed 3
thu 3
tik 3
sunvalley 3
hawaii 3
evacs 3
gavin 3
bupkg 3
survivors 3
surf 3
converter 3
calfor 3
austin 3
test 3
epa 3
telegraph 3
fun 3
5p 3
cleveland 3
mckinney 3
letter 3
closure 3
auburn 3
conditions 3
bailey 3
buy 3
incscribe 3
campbell 3
caitlyn 3
crimes 3
ahmaud 3
decorations 3
de 3
atum 3
confrontation 3
adeline 3
mariposa 3
med 3
kiosk 3
heroes 3
covered 3
body 3
alcatraz 3
dispensary 3
2019 3
50 3
711 3
aaron 3
idaho 3
friends 3
beaten 3
fitz 3
guidelines 3
logs 3
discuss 3
flotus 3
g

In [13]:
#all_text =pd.Series(( ' '.join(list(qtr_df["StoryText"]))))
all_text = []

custom_stop_words = ['0','1','2','3','4','5','6','7','8','9','\'','"','\n','\t',',','.',',','-',':']

for row in qtr_df.iterrows():
    #loop_text = ' '.join(list(row[1]['StoryText']))
    loop_text = row[1]['StoryText']
    
    for word in custom_stop_words:
        loop_text = loop_text.replace(word," ")
    
    all_text.append(loop_text)
    
tfIdfVectorizer=TfidfVectorizer(ngram_range=(2,2),use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(all_text)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(50))


                     TF-IDF
save communities   0.243044
optimism from      0.243044
firefighters as    0.243044
increase sense     0.243044
they dig           0.243044
lines increase     0.243044
dig in             0.232595
of optimism        0.232595
from firefighters  0.225182
the containment    0.199114
containment lines  0.199114
communities in     0.191121
area from          0.189458
tahoe area         0.187896
the tahoe          0.185030
in to              0.167168
start to           0.167168
sense of           0.162244
to save            0.154140
to start           0.140490
as they            0.134310
the caldor         0.131958
see the            0.131444
caldor fire        0.131275
re going           0.129173
quote we           0.118725
to see             0.107428
going to           0.094845
we re              0.089499
from the           0.087146
in the             0.053094
quiet champion     0.000000
quiet at           0.000000
quiet storm        0.000000
quiet maxwell      0

In [14]:
#//***************************************
#//*** Apply Common Cleanup operations
#//***************************************
#//*** In anticpation that I'll be re-using text cleanup code. I'm adding some robustness to the function.
#//*** Adding kwargs to disable features that default to true.
#//*** Whether an action is skipped or executed is based on a boolean value stored in action_dict.
#//*** Key values will default to true. If code needs to be defaulted to False, a default_false list can be added later
#//*** All Boolean kwarg keya are stored in kwarg list. This speeds up the coding of the action_dict.
#//*** As Kwargs are added 
def mr_clean_text(input_series, input_options={}):
    
    #//*** import time library
    try:
        type(time)
    except:
        import time
    
    #//*** Start Timing the process
    start_time = time.time()
    
    #//*** Add some data validation. I'm preparing this function for additional use. I'm checking if future users (ie future me)
    #//*** may throw some garbage at this function. Experience has taught me to fail safely wherever possible.

    #//*** All kwargs are listed here. These initialize TRUE by default.
    key_list = [ "lower", "newline", "html", "remove_empty", "punctuation" ]
    
    #//*** Build Action Dictionary
    action_dict = { } 
    
    #//*** Build the keys from kwarg_list and default them to TRUE
    for key in key_list:
        action_dict[key] = True
        
    #//*** Loop through the input kwargs (if any). Assign the action_dict values based on the kwargs:
    for key,value in input_options.items():
        print(key,value)
        action_dict[key] = value
    
    
    #//*************************************************************************
    #//*** The Cleanup/Processing code is a straight lift from DSC550 - Week02
    #//*************************************************************************
    #//*** Convert to Lower Case, Default to True
    if action_dict["lower"]:
        input_series = input_series.str.lower()
    
   
    #//*** Remove New Lines
    if action_dict["newline"]:
        #//*** Rmove \r\n
        input_series = input_series.str.replace(r'\r?\n',"")

        #//*** Remove \n new lines
        input_series = input_series.str.replace(r'\n',"")

    #//*** Remove html entities, observed entities are &gt; and &lt;. All HTML entities begin with & and end with ;.
    #//*** Let's use regex to remove html entities
    if action_dict["html"]:
        input_series = input_series.str.replace(r'&.*;',"")

    #//*** Remove the empty lines
    if action_dict["remove_empty"]:
        input_series = input_series[ input_series.str.len() > 0]

    #//*** Remove punctuation
    if action_dict["punctuation"]:
        #//*** Load libraries for punctuation if not already loaded.
        #//*** Wrapping these in a try, no sense in importing libraries that already exist.
        #//*** Unsure of the cost of reimporting libraries (if any). But testing if library is already loaded feels
        #//*** like a good practice
        try:
            type(sys)
        except:
            import sys

        try:
            type(unicodedata)
        except:
            import unicodedata
        
        #//*** replace Comma and Period with a space.
        for punct in [",",".","$"]:
            input_series = input_series.str.replace(punct," ")

        #//*** Remove punctuation using the example from the book
        punctuation = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P') )
        input_series = input_series.str.translate(punctuation)

    print(f"Text Cleaning Time: {time.time() - start_time}")

    return input_series
#//*** Remove Stop words from the input list
def remove_stop_words(input_series):
    
    #//*** This function removes stop_words from a series.
    #//*** Works with series.apply()
    def apply_stop_words(input_list):

        #//*** Load Stopwords   
        for word in input_list:
            if word in stop_words:
                input_list.remove(word)
        return input_list

    #//*** import nltk if needed
    try:
        type(nltk)
    except:
        import nltk
        
    stopwords = nltk.corpus.stopwords

    #//*** Stopwords requires an additional download
    try:
        type(stopwords)
    except:
        nltk.download('stopwords')


    #//*** import time library
    try:
        type(time)
    except:
        import time

    #//*** Start Timing the process
    start_time = time.time()


    #//*** The stop_words include punctuation. Stop Word Contractions will not be filtered out.
    stop_words = []

    #//*** Remove apostrophies from the stop_words
    for stop in stopwords.words('english'):
        stop_words.append(stop.replace("'",""))

    
    #//*** Remove Stop words from the tokenized strings in the 'process' column
    #input_series = input_series.apply(remove_stop_words,stop_words)
    
    input_series = input_series.apply(apply_stop_words)

    print(f"Stop Words Time: {time.time() - start_time}")
    
    return input_series
#//*** Tokenize a Series containing Strings.
#//*** Breaking this out into it's own function for later reuse.
#//*** Not a lot of code here, but it helps to keep the libraries localized. This creates standarization for future
#//*** Stoneburner projects. Also has the ability to add functionality as needed.

def tokenize_series(input_series):
    
    try:
        type(nltk)
    except:
        import nltk
    
    word_tokenize = nltk.tokenize.word_tokenize 
    
    #//*** import time library
    try:
        type(time)
    except:
        import time
    
    #//*** Start Timing the process
    start_time = time.time()
    try:
        input_series = input_series.apply(word_tokenize)
    except:
        #//*** Try again is punkt not downloaded
        nltk.download('punkt')
        input_series = input_series.apply(word_tokenize)
        
    
    print(f"Tokenize Time: {time.time() - start_time}")
    
    return input_series




In [36]:
qtr_df['StoryText'] = qtr_df['StoryText'].str.lower()
babba_df = qtr_df[qtr_df['StoryText'].str.contains("building a better bay area")].copy()
babba_df

#qtr_df[qtr_df['StoryText'].str.contains("race and culture")]['StoryText'].iloc[0]

race_df = qtr_df[qtr_df['StoryText'].str.contains("race and social justice")]
race_df

education_df = qtr_df[qtr_df['StoryText'].str.contains("education reporter")]
education_df

oys_df = qtr_df[qtr_df['StoryText'].str.contains("seven on your side")]
oys_df

covid_df = qtr_df[qtr_df['StoryText'].str.contains("covid")]
covid_df = covid_df[covid_df['storyslug'].str.contains("PKG")]
covid_df


climate_df = qtr_df[qtr_df['StoryText'].str.contains("climate")]
climate_df = climate_df[climate_df['storyslug'].str.contains("PKG")]
climate_df

iteam_df = qtr_df[qtr_df['StoryText'].str.contains("i-team")]
iteam_df = qtr_df[(qtr_df['StoryText'].str.contains("melanie woodrow")) | (qtr_df['StoryText'].str.contains("dan noyes"))]
iteam_df = iteam_df[iteam_df['storyslug'].str.contains("PKG")]
#iteam_df.loc[3501]['StoryText']
iteam_df
#tdf[tdf['duration']> 60]


,start_date,rundown,title_id,storyslug,duration,StoryText
209,2021-09-02,5PM Weekday,32591116,5P UCSF DOCTOR PKG,138,(anchor: as the san francisco board of educati...
1788,2021-09-16,4PM Weekday,32826770,4P COCO SCHOOL VACCINE PKG,214,(anchor: the contra costa county health office...
1824,2021-09-17,6PM Weekday,32827622,6P AFGHAN HELP PKG,114,((coldopen vo: one cell phone image tells the ...
1896,2021-09-18,4PM Weekday,32982082,4P RELIGIOUS EXEMPTION PKG,132,((anchor: more than half of san francisco's un...
2086,2021-09-21,4PM Weekday,32881173,4P OAK CRIME OVERVIEW PKG,124,": for the second consecutive year, the city of..."
2197,2021-09-22,5PM Weekday,32902361,5P BAYVIEW RV PROBLEM PKG,139,"(anchor: human waste, garbage and needles .. t..."
2505,2021-09-24,5PM Weekday,32939510,5P ZOGG FIRE PKG,130,"(anchor: nearly one year ago, the zogg fire cl..."
2793,2021-09-26,5PM Weekday,32998656,5P HATE CRIME ARREST PKG,168,(anchor: the san francisco district attorney a...
2906,2021-09-27,4PM Weekday,33016678,4P COVID AIR TRAVEL PKG,147,(anchor: travelers could be looking at new cov...
2921,2021-09-27,5PM Weekday,33016429,5P COVID AIR TRAVEL PKG,147,(anchor: travelers could be looking at new cov...
